In [6]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

In [9]:
# Import existing api configuration code
import sys
sys.path.append('../trading')

In [10]:
from config import * 

ACCOUNT STATUS IS ACTIVE


In [11]:
polygon = api.polygon

In [17]:
def fetch_dividends(sym):
    """Given a symbol returns a dataframe of dividends"""
    symbol_dividends = polygon.dividends(sym)
    fields = ['amount', 'declaredDate', 'exDate', 'indicated', 'paymentDate', 'qualified', 'recordDate', 'symbol', 'type']
    # Initialize general data structure
    output = {}
    for field in fields:
        output[field] = []
    
    for dividend in symbol_dividends:
        for field in fields:
            converted = dividend._raw
            try:
                output[field].append(converted[field])
            except:
                output[field].append(np.nan)
            
    df = pd.DataFrame.from_dict(output)
    for date_field in [x for x in fields if 'Date' in x]:
        df[date_field] = pd.to_datetime(df[date_field])
    return df 

def fetch_stock(sym, limit = 499):
    """Fetches limit entries for a given stock symbol and puts it in a dataframe"""
    agg = polygon.historic_agg('day', sym, limit=limit).df
    dates = [str(x).split("T")[0] for x in agg.index.values]
    agg['formatted_dates'] = pd.to_datetime(dates)
    return agg

def fetch_dividend_pricing(sym, limit = 499):
    """Returns relevant price characteristics for a dividend stock over some period of time
    Includes
    Exdate - Ex dividend date 
    amount - The amount of the dividend output
    open   - The opening price of the stock
    high   - The high price of the stock that day 
    low    - The low price of the stock that day 
    close  - The closing price of the stock that day    
    """
    a_dat = fetch_stock(sym, limit = limit)
    a_div = fetch_dividends(sym)
    joining_a_dat = a_dat[['open', 'high', 'low', 'close', 'formatted_dates']]
    joining_a_dat = joining_a_dat.reset_index().drop(['day'], axis=1).set_index('formatted_dates')
    joining_a_div = a_div.drop(['type','qualified','indicated', 'paymentDate', 'recordDate', 'symbol', 'declaredDate'], axis=1).set_index('exDate')
    joined_data = joining_a_div.join(joining_a_dat)
    return joined_data[pd.notnull(joined_data.open)]
    
    

In [28]:
agg = fetch_stock("aapl")

In [31]:
agg = polygon.historic_agg('day', "appl", limit=1000).df

In [18]:
appl_dat = fetch_stock("AAPL")
appl_div = fetch_dividends("AAPL")

In [19]:
appl_dat = fetch_dividend_pricing("AAPL")

In [21]:
appl_dat.index.values

array(['2018-11-08T00:00:00.000000000', '2018-08-10T00:00:00.000000000',
       '2018-05-11T00:00:00.000000000', '2018-02-09T00:00:00.000000000',
       '2017-11-10T00:00:00.000000000', '2017-08-10T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [22]:
appl_dat.index[0] - pd.Timedelta(days = 2)

Timestamp('2018-11-06 00:00:00')

In [23]:
# TODO Find a way to determine whether the stock recovers well post-dividend distribution